In [64]:
import pandas as pd
import numpy as np
import boto3
import psycopg2
import configparser

In [65]:
config = configparser.ConfigParser()
config.read('proy_escec.cfg')

['proy_escec.cfg']

In [66]:
RDS_HOST = 'proyecto.cf32hjwhygka.us-east-1.rds.amazonaws.com'
#RDS_HOST=config.get('DW', 'RDS_HOST')
#RDS_HOST
print(RDS_HOST)

proyecto.cf32hjwhygka.us-east-1.rds.amazonaws.com


Conexion a la base de datos de Postgres

In [67]:
postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""  

##### Creamos Dimensión de Clientes

In [68]:
sql_query = 'SELECT * FROM cliente;'
df_cliente = pd.read_sql(sql_query, postgres_driver)
dimcliente = df_cliente
dimcliente.head()

,id_cliente,nombre,apellido,correo,telefono,direccion
0,25129746,Randall,Griffin,hernandezhenry@yahoo.com,459-29-8362,None
1,92183851,Timothy,Bruce,paul14@yahoo.com,728-67-1921,None
2,57023133,Harry,Atkinson,elizabeth50@gmail.com,700-71-8791,None
3,68683279,Daniel,Edwards,jessica16@yahoo.com,774-91-3965,None
4,30639326,Andrew,Bailey,bensonian@hotmail.com,568-28-9506,None


#### Dimension de Sucursal

In [69]:
sql_query = 'SELECT * FROM sucursal;'
df_sucursal = pd.read_sql(sql_query, postgres_driver)
dimsucursal = df_sucursal
dimsucursal.head()

,id_sucursal,sucursal
0,50,zona 10
1,51,Centro civico
2,52,Peten
3,53,Quetzaltenango


Dimension Libros

In [70]:
sql_query = 'SELECT * FROM libros'
df_libros = pd.read_sql(sql_query, postgres_driver)
df_libros.head()

,id_libro,titulo,id_lenguaje,id_genero,id_editorial,id_autor
0,39547,Kimberly Barnett,11,21,41,31
1,18929,Kimberly Levine,11,20,41,31
2,93123,Matthew Young,12,23,40,32
3,70228,Timothy Scott,12,22,40,34
4,19679,Kiara Bailey,11,20,41,30


In [71]:
sql_query = 'SELECT * FROM lenguaje; '
df_lenguaje = pd.read_sql(sql_query, postgres_driver)
df_lenguaje.head()

,id_lenguaje,lenguaje
0,10,español
1,11,ingles
2,12,portugues


In [72]:
df_lible = df_libros.merge(df_lenguaje, on = 'id_lenguaje', how= 'inner')
df_lible.head()

,id_libro,titulo,id_lenguaje,id_genero,id_editorial,id_autor,lenguaje
0,39547,Kimberly Barnett,11,21,41,31,ingles
1,18929,Kimberly Levine,11,20,41,31,ingles
2,19679,Kiara Bailey,11,20,41,30,ingles
3,48781,Jason Anderson,11,22,41,33,ingles
4,97567,John Cook,11,23,41,31,ingles


In [73]:
sql_query = 'SELECT * FROM genero; '
df_genero = pd.read_sql(sql_query, postgres_driver)
df_genero.head()

,id_genero,genero
0,20,suspenso
1,21,terror
2,22,superacion
3,23,novela


In [74]:
df_liblege = df_lible.merge(df_genero, on='id_genero', how = 'inner')
df_liblege.head()

,id_libro,titulo,id_lenguaje,id_genero,id_editorial,id_autor,lenguaje,genero
0,39547,Kimberly Barnett,11,21,41,31,ingles,terror
1,95437,Derek Ortiz,11,21,40,31,ingles,terror
2,77053,Rhonda Norton,11,21,42,31,ingles,terror
3,63372,Lisa Fry,11,21,43,33,ingles,terror
4,95529,Jennifer Jones,11,21,41,31,ingles,terror


In [75]:
sql_query = 'SELECT * FROM editorial; '
df_editorial = pd.read_sql(sql_query, postgres_driver)
df_editorial.head()

,id_editorial,editorial
0,40,Libros Rotos
1,41,Reflejos
2,42,Profesional
3,43,Educativa


In [76]:
df_liblegeedi = df_liblege.merge(df_editorial, on='id_editorial', how ='inner')
df_liblegeedi.head()

,id_libro,titulo,id_lenguaje,id_genero,id_editorial,id_autor,lenguaje,genero,editorial
0,39547,Kimberly Barnett,11,21,41,31,ingles,terror,Reflejos
1,95529,Jennifer Jones,11,21,41,31,ingles,terror,Reflejos
2,89430,Whitney Lee,11,21,41,33,ingles,terror,Reflejos
3,89197,Caleb Li,11,21,41,30,ingles,terror,Reflejos
4,67893,Lisa Ward,11,21,41,30,ingles,terror,Reflejos


In [77]:
sql_query = 'SELECT * FROM autor; '
df_autor = pd.read_sql(sql_query, postgres_driver)
df_autor.head()

,id_autor,autor
0,30,Juan Soto
1,31,Susana Rojas
2,32,Pedro Fernandez
3,33,Rocio Chacon
4,34,Julio Quezada


In [78]:
df_liblegeediau = df_liblegeedi.merge(df_autor, on='id_autor', how ='inner')
df_liblegeediau.head()

,id_libro,titulo,id_lenguaje,id_genero,id_editorial,id_autor,lenguaje,genero,editorial,autor
0,39547,Kimberly Barnett,11,21,41,31,ingles,terror,Reflejos,Susana Rojas
1,95529,Jennifer Jones,11,21,41,31,ingles,terror,Reflejos,Susana Rojas
2,54835,Michael Best,11,21,41,31,ingles,terror,Reflejos,Susana Rojas
3,56821,Ronnie Leonard,12,21,41,31,portugues,terror,Reflejos,Susana Rojas
4,39603,Robert Jordan,10,21,41,31,español,terror,Reflejos,Susana Rojas


In [79]:
dim_libros = df_liblegeediau
dim_libros.drop(['id_lenguaje', 'id_genero', 'id_editorial', 'id_autor'], axis=1, inplace=True)
dim_libros.head()

,id_libro,titulo,lenguaje,genero,editorial,autor
0,39547,Kimberly Barnett,ingles,terror,Reflejos,Susana Rojas
1,95529,Jennifer Jones,ingles,terror,Reflejos,Susana Rojas
2,54835,Michael Best,ingles,terror,Reflejos,Susana Rojas
3,56821,Ronnie Leonard,portugues,terror,Reflejos,Susana Rojas
4,39603,Robert Jordan,español,terror,Reflejos,Susana Rojas


Dimension de Tiempo

In [80]:
sql_query = 'SELECT fecha_hora FROM orden;'
dimFechaHora = pd.read_sql(sql_query, postgres_driver)
dimFechaHora.head()

,fecha_hora
0,2023-03-30 20:54:11
1,2023-03-11 17:21:47
2,2023-04-05 03:48:58
3,2023-02-09 11:32:02
4,2023-01-21 08:34:13


In [81]:
dimFechaHora['year'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).year
dimFechaHora['month'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).month
dimFechaHora['quarter'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).quarter
dimFechaHora['day'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).day
dimFechaHora['week'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).week
dimFechaHora['dayofweek'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).dayofweek
dimFechaHora['hour'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).hour
dimFechaHora['minute'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).minute
dimFechaHora['second'] = pd.DatetimeIndex(dimFechaHora ['fecha_hora']).second
dimFechaHora['is_weekend'] = dimFechaHora['dayofweek'].apply(lambda x: 1 if x > 5 else 0)
dimFechaHora.head()

C:\Users\juanm\AppData\Local\Temp\ipykernel_17812\1225628087.py:5: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  dimFechaHora['week'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).week


,fecha_hora,year,month,quarter,day,week,dayofweek,hour,minute,second,is_weekend
0,2023-03-30 20:54:11,2023,3,1,30,13,3,20,54,11,0
1,2023-03-11 17:21:47,2023,3,1,11,10,5,17,21,47,0
2,2023-04-05 03:48:58,2023,4,2,5,14,2,3,48,58,0
3,2023-02-09 11:32:02,2023,2,1,9,6,3,11,32,2,0
4,2023-01-21 08:34:13,2023,1,1,21,3,5,8,34,13,0


In [82]:
dimFechaHora['id_fecha'] = dimFechaHora['year'].astype(str) + dimFechaHora['month'].astype(str)
dimFechaHora['id_fecha'] = dimFechaHora['id_fecha'].astype(str) + dimFechaHora['day'].astype(str)
dimFechaHora['id_fecha'] = dimFechaHora['id_fecha'].astype(str) + dimFechaHora['hour'].astype(str)
dimFechaHora['id_fecha'] = dimFechaHora['id_fecha'].astype(str) + dimFechaHora['minute'].astype(str)
dimFechaHora['id_fecha'] = dimFechaHora['id_fecha'].astype(str) + dimFechaHora['second'].astype(str)
dimFechaHora.drop_duplicates(inplace=True)
dimFechaHora.head()

,fecha_hora,year,month,quarter,day,week,dayofweek,hour,minute,second,is_weekend,id_fecha
0,2023-03-30 20:54:11,2023,3,1,30,13,3,20,54,11,0,2023330205411
1,2023-03-11 17:21:47,2023,3,1,11,10,5,17,21,47,0,2023311172147
2,2023-04-05 03:48:58,2023,4,2,5,14,2,3,48,58,0,20234534858
3,2023-02-09 11:32:02,2023,2,1,9,6,3,11,32,2,0,20232911322
4,2023-01-21 08:34:13,2023,1,1,21,3,5,8,34,13,0,202312183413


Table de hechos de Orden

In [83]:
sql_query = 'SELECT * FROM orden; '
df_orden = pd.read_sql(sql_query, postgres_driver)
df_orden.head()

,id_orden,fecha_hora,id_libro,id_sucursal,id_cliente
0,0,2023-03-30 20:54:11,48027,52,54780015
1,1,2023-03-11 17:21:47,79232,51,87766421
2,2,2023-04-05 03:48:58,63700,52,75568309
3,3,2023-02-09 11:32:02,81097,50,55021606
4,4,2023-01-21 08:34:13,11329,51,30509395


In [84]:
df_orden_libro = df_orden.merge(dim_libros, on = 'id_libro', how= 'inner')
df_orden_libro.head()

,id_orden,fecha_hora,id_libro,id_sucursal,id_cliente,titulo,lenguaje,genero,editorial,autor
0,0,2023-03-30 20:54:11,48027,52,54780015,Tracy Stone,español,novela,Libros Rotos,Julio Quezada
1,1,2023-03-11 17:21:47,79232,51,87766421,Frederick Robles,ingles,superacion,Profesional,Juan Soto
2,31,2023-03-22 09:57:02,79232,50,50277606,Frederick Robles,ingles,superacion,Profesional,Juan Soto
3,2,2023-04-05 03:48:58,63700,52,75568309,Dr. Crystal Kelley,español,terror,Libros Rotos,Juan Soto
4,3,2023-02-09 11:32:02,81097,50,55021606,Raymond Clark,ingles,novela,Educativa,Juan Soto


In [87]:
df_orden_libro['year'] = pd.DatetimeIndex(df_orden_libro['fecha_hora']).year
df_orden_libro['month'] = pd.DatetimeIndex(df_orden_libro['fecha_hora']).month
df_orden_libro['day'] = pd.DatetimeIndex(df_orden_libro['fecha_hora']).day
df_orden_libro['hour'] = pd.DatetimeIndex(df_orden_libro['fecha_hora']).hour
df_orden_libro['minute'] = pd.DatetimeIndex(df_orden_libro['fecha_hora']).minute
df_orden_libro['second'] = pd.DatetimeIndex(df_orden_libro['fecha_hora']).second
df_orden_libro['id_fecha'] = df_orden_libro['year'].astype(str) + df_orden_libro['month'].astype(str)
df_orden_libro['id_fecha'] = df_orden_libro['id_fecha'].astype(str) + df_orden_libro['day'].astype(str)
df_orden_libro['id_fecha'] = df_orden_libro['id_fecha'].astype(str) + df_orden_libro['hour'].astype(str)
df_orden_libro['id_fecha'] = df_orden_libro['id_fecha'].astype(str) + df_orden_libro['minute'].astype(str)
df_orden_libro['id_fecha'] = df_orden_libro['id_fecha'].astype(str) + df_orden_libro['second'].astype(str)
df_orden_libro.drop(['year', 'month', 'day', 'hour', 'minute', 'fecha_hora'], axis=1)
df_orden_libro.head()

,id_orden,fecha_hora,id_libro,id_sucursal,id_cliente,titulo,lenguaje,genero,editorial,autor,year,month,day,hour,minute,id_fecha,second
0,0,2023-03-30 20:54:11,48027,52,54780015,Tracy Stone,español,novela,Libros Rotos,Julio Quezada,2023,3,30,20,54,2023330205411,11
1,1,2023-03-11 17:21:47,79232,51,87766421,Frederick Robles,ingles,superacion,Profesional,Juan Soto,2023,3,11,17,21,2023311172147,47
2,31,2023-03-22 09:57:02,79232,50,50277606,Frederick Robles,ingles,superacion,Profesional,Juan Soto,2023,3,22,9,57,20233229572,2
3,2,2023-04-05 03:48:58,63700,52,75568309,Dr. Crystal Kelley,español,terror,Libros Rotos,Juan Soto,2023,4,5,3,48,20234534858,58
4,3,2023-02-09 11:32:02,81097,50,55021606,Raymond Clark,ingles,novela,Educativa,Juan Soto,2023,2,9,11,32,20232911322,2


In [88]:
df_orden_libro.head()

,id_orden,fecha_hora,id_libro,id_sucursal,id_cliente,titulo,lenguaje,genero,editorial,autor,year,month,day,hour,minute,id_fecha,second
0,0,2023-03-30 20:54:11,48027,52,54780015,Tracy Stone,español,novela,Libros Rotos,Julio Quezada,2023,3,30,20,54,2023330205411,11
1,1,2023-03-11 17:21:47,79232,51,87766421,Frederick Robles,ingles,superacion,Profesional,Juan Soto,2023,3,11,17,21,2023311172147,47
2,31,2023-03-22 09:57:02,79232,50,50277606,Frederick Robles,ingles,superacion,Profesional,Juan Soto,2023,3,22,9,57,20233229572,2
3,2,2023-04-05 03:48:58,63700,52,75568309,Dr. Crystal Kelley,español,terror,Libros Rotos,Juan Soto,2023,4,5,3,48,20234534858,58
4,3,2023-02-09 11:32:02,81097,50,55021606,Raymond Clark,ingles,novela,Educativa,Juan Soto,2023,2,9,11,32,20232911322,2


In [89]:
df_orden_libro_suc = df_orden_libro.merge(df_sucursal, on = 'id_sucursal', how= 'inner')
df_orden_libro_suc.head()

,id_orden,fecha_hora,id_libro,id_sucursal,id_cliente,titulo,lenguaje,genero,editorial,autor,year,month,day,hour,minute,id_fecha,second,sucursal
0,0,2023-03-30 20:54:11,48027,52,54780015,Tracy Stone,español,novela,Libros Rotos,Julio Quezada,2023,3,30,20,54,2023330205411,11,Peten
1,2,2023-04-05 03:48:58,63700,52,75568309,Dr. Crystal Kelley,español,terror,Libros Rotos,Juan Soto,2023,4,5,3,48,20234534858,58,Peten
2,184,2023-03-19 01:40:02,11329,52,87792025,Jennifer Orozco,ingles,suspenso,Educativa,Juan Soto,2023,3,19,1,40,20233191402,2,Peten
3,5,2023-04-01 20:21:58,70113,52,64837260,Jenna Sullivan,portugues,suspenso,Reflejos,Julio Quezada,2023,4,1,20,21,202341202158,58,Peten
4,70,2023-01-04 11:51:17,70113,52,27790553,Jenna Sullivan,portugues,suspenso,Reflejos,Julio Quezada,2023,1,4,11,51,202314115117,17,Peten


In [112]:
df_orden_libro_suc_cliente = df_orden_libro_suc.merge(df_cliente, on = 'id_cliente', how= 'inner')
df_orden_libro_suc_cliente.drop(['id_libro', 'id_sucursal', 'id_cliente', 'id_fecha', 'year', 'month', 'day', 'hour', 'minute', 'second', 'direccion'], axis=1, inplace=True)
fact_orden=df_orden_libro_suc_cliente
fact_orden.head()

,id_orden,fecha_hora,titulo,lenguaje,genero,editorial,autor,sucursal,nombre,apellido,correo,telefono
0,0,2023-03-30 20:54:11,Tracy Stone,español,novela,Libros Rotos,Julio Quezada,Peten,Kristin,Dixon,laurahernandez@yahoo.com,180-88-7815
1,2,2023-04-05 03:48:58,Dr. Crystal Kelley,español,terror,Libros Rotos,Juan Soto,Peten,Nicole,Pope,rachel24@yahoo.com,660-67-5211
2,184,2023-03-19 01:40:02,Jennifer Orozco,ingles,suspenso,Educativa,Juan Soto,Peten,Robert,Hanson,hstrong@yahoo.com,833-18-4581
3,5,2023-04-01 20:21:58,Jenna Sullivan,portugues,suspenso,Reflejos,Julio Quezada,Peten,Frances,Bennett,fwhite@hotmail.com,789-71-8289
4,70,2023-01-04 11:51:17,Jenna Sullivan,portugues,suspenso,Reflejos,Julio Quezada,Peten,Julia,Townsend,ngiles@hotmail.com,553-85-9351


Lectura de archivo desde S3

In [91]:
s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-1',
    aws_access_key_id = config.get('IAM', 'ACCESS_KEY'),
    aws_secret_access_key = config.get('IAM', 'SECRET_ACCESS_KEY')
)

In [92]:
for bucket in s3.buckets.all():
    S3_BUCKET_NAME = bucket.name
    print(bucket.name)

mibucket-cdp-9710120


In [93]:
S3_BUCKET_NAME = 'mibucket-cdp-9710120'

In [94]:
#extraemos todo lo que está en el bucket
remoteFileList = []
for objt in s3.Bucket(S3_BUCKET_NAME).objects.all():
    remoteFileList.append(objt.key)

remoteFileList

['calificacion.xlsx']

Leemos la Data desde S3

In [95]:
import io

for remoteFile in remoteFileList:
    try:
        file = s3.Bucket(S3_BUCKET_NAME).Object(remoteFile).get()
        data = file['Body'].read()
        df_calificacion = pd.read_excel(io.BytesIO(data), engine='openpyxl')
    except Exception as ex:
        print("No es un archivo.")
        print(ex)

df_calificacion.head()

,Calificacion,Codigo
0,Excelente,A
1,Bueno,B
2,Regular,C
3,Malo,D


Agregamos nueva columna con Calificaciones

In [ ]:
sql_query = '''SELECT id_libro, FROM libros;'''
df_calificacion = pd.read_sql(sql_query, postgres_driver)
df_calificacion.head()

##### Creación de Instancias en AWS para DW

In [97]:
config = configparser.ConfigParser()
config.read('proy_escec.cfg')

['proy_escec.cfg']

In [98]:
aws_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                    aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                    region_name='us-east-1')
print(aws_conn)

In [99]:
rdsInstanceIds = []

response = aws_conn.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['dw', 'proyecto']


In [100]:
rdsIdentifier = 'dw'

In [101]:
try:
    response = aws_conn.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS_POSTGRESDW', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="postgres",
            MasterUsername=config.get('RDS_POSTGRESDW', 'DB_USER'),
            MasterUserPassword=config.get('RDS_POSTGRESDW', 'DB_PASSWORD'),
            Port=int(config.get('RDS_POSTGRESDW', 'DB_PORT')),
            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
            PubliclyAccessible=True
        )
    print(response)
except aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

La Instancia de Base de Datos ya Existe.


In [102]:
try:
     instances = aws_conn.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_DW_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_DW_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

dw.cf32hjwhygka.us-east-1.rds.amazonaws.com


In [103]:
import Proy_create_dw_query

try:
    db_conn = psycopg2.connect(
        database=config.get('RDS_POSTGRESDW', 'DB_NAME'), 
        user=config.get('RDS_POSTGRESDW', 'DB_USER'),
        password=config.get('RDS_POSTGRESDW', 'DB_PASSWORD'), 
        host=RDS_DW_HOST,
        port=config.get('RDS_POSTGRESDW', 'DB_PORT')
    )

    cursor = db_conn.cursor()
    cursor.execute(Proy_create_dw_query.CREATE_DW)
    db_conn.commit()
    print("DW Creada Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)

ERROR: Error al crear la base de datos.
relation "dimlibros" already exists



In [104]:
db_host = 'dw.cf32hjwhygka.us-east-1.rds.amazonaws.com'
db_port = '5432'
db_name = 'dw'
db_user = 'postgres'
db_password = 'KapldrE7hlSpAFuBeredEyagafR!M'

postgres_driver = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"



In [105]:
dimcliente.to_sql('dimclientes', postgres_driver, index=False, if_exists='append')

24

In [107]:
dimFechaHora.to_sql('dimfechahora', postgres_driver,index=False, if_exists='append')

193

In [47]:
dim_libros.to_sql('dimlibros',postgres_driver, index=False, if_exists='append')

401

In [48]:
dimsucursal.to_sql('dimsucursal', postgres_driver, index=False, if_exists='append')

4

In [113]:
fact_orden.to_sql('factorden', postgres_driver, index=False, if_exists='append')

193